# 에지검출을 이용한 마스크 검출 정확도 높이기
마스크 검출을 하기 전에 소벨 마스크로 윤곽선만 추출해서 인식 정확 높이기

In [4]:
import os
import cv2
import numpy as np

In [7]:
default_data_path = './data/learning/'

## 윤곽선 추출 이미지 가져오기
B,G,R 각 채널들을 각각 소벨 마스크로 윤곽선 추출 후 다시 결합

In [1]:
def getSobelImage(mode, size):
    images = []
    labels = []
    for idx, now in enumerate(['Mask', 'Non Mask']):
        default_path = os.path.join(default_data_path, mode, now)
        img_name_list = os.listdir(default_path)

        for img_name in img_name_list:
            img = cv2.imread(os.path.join(default_path, img_name)) # 이미지 읽기
            img = cv2.resize(img, size) # 이미지 224*224로 리사이즈
            channels = cv2.split(img) # B,G,R 채널 분리
            
            # 각 채널마다 소벨 마스크 적용
            for i, c in enumerate(channels):
                channels = list(channels)
                dx = cv2.Sobel(c, cv2.CV_64F, 1, 0, ksize=3) # x축 적용
                dx = cv2.convertScaleAbs(dx)
                dy = cv2.Sobel(c, cv2.CV_64F, 0, 1, ksize=3) # y축 적용
                dy = cv2.convertScaleAbs(dy)
                c = cv2.addWeighted(dx, 1, dy, 1, 0) # x, y축 결합
                channels[i] = c
            img = cv2.merge(channels) # 채널 병합
            images.append(img)
            labels.append(idx)
    return np.asarray(images) / 225, np.asarray(labels) # 각 채널 225로 나누어 정규화

In [4]:
train_data_sobel, train_y_sobel = getSobelImage('Train', (224,224))
val_data_sobel, val_y_sobel = getSobelImage('Validation', (224,224))
train_data_sobel

array([[[[0.        , 0.        , 0.        ],
         [0.72888889, 0.90666667, 0.91555556],
         [0.71111111, 0.78222222, 0.8       ],
         ...,
         [0.        , 0.        , 0.        ],
         [0.01777778, 0.00888889, 0.01777778],
         [0.        , 0.        , 0.        ]],

        [[0.00888889, 0.        , 0.00888889],
         [0.73777778, 0.90666667, 0.92444444],
         [0.71111111, 0.78222222, 0.80888889],
         ...,
         [0.01777778, 0.01777778, 0.01777778],
         [0.03555556, 0.02666667, 0.03555556],
         [0.01777778, 0.01777778, 0.01777778]],

        [[0.        , 0.        , 0.00888889],
         [0.72888889, 0.90666667, 0.92444444],
         [0.70222222, 0.78222222, 0.81777778],
         ...,
         [0.01777778, 0.01777778, 0.01777778],
         [0.03555556, 0.02666667, 0.03555556],
         [0.01777778, 0.01777778, 0.01777778]],

        ...,

        [[0.02666667, 0.02666667, 0.02666667],
         [0.22222222, 0.18666667, 0.13333333]

## 기학습된 VGG16 전이학습
VGG16모델 부분은 가중치를 고정시켜두고, 출력부분에 fully connected layer를 추가해 이 부분만 학습

In [5]:
import tensorflow.keras as keras

In [6]:
def get_vgg16():
    # VGG16모델 불러옴
    pre_trained_vgg = keras.applications.VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    pre_trained_vgg.trainable = False # weight 고정
    
    # fully connected 모델 구현
    model = keras.models.Sequential()
    model.add(pre_trained_vgg)
    model.add(keras.layers.Flatten())
    model.add(keras.layers.Dense(1024, kernel_regularizer=keras.regularizers.l1_l2(l1=0.001,l2=0.001), activation='relu'))
    model.add(keras.layers.Dropout(0.5))
    model.add(keras.layers.Dense(512, kernel_regularizer=keras.regularizers.l1_l2(l1=0.001,l2=0.001), activation='relu'))
    model.add(keras.layers.Dropout(0.5))
    model.add(keras.layers.Dense(1, activation='sigmoid')) # 0, 1 두 값으로 나오게 sigmoid 함수 사용
    
    return model

In [8]:
model_sobel = get_vgg16()

## 컴파일 후 학습
모델 컴파일 후 학습 진행

In [10]:
model_sobel.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])
model_sobel.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 7, 7, 512)         14714688  
                                                                 
 flatten_1 (Flatten)         (None, 25088)             0         
                                                                 
 dense_3 (Dense)             (None, 1024)              25691136  
                                                                 
 dropout_2 (Dropout)         (None, 1024)              0         
                                                                 
 dense_4 (Dense)             (None, 512)               524800    
                                                                 
 dropout_3 (Dropout)         (None, 512)               0         
                                                                 
 dense_5 (Dense)             (None, 1)                

In [11]:
callbacks = [keras.callbacks.EarlyStopping(monitor='accuracy', mode='max', patience=5),
                 keras.callbacks.ModelCheckpoint(filepath='./data/model/best_model_sobel.h5', monitor='accuracy', save_best_only=True)]
model_sobel.fit(train_data_sobel, train_y_sobel, epochs=50, batch_size=8,
                    validation_data=(val_data_sobel, val_y_sobel), shuffle=True, callbacks=callbacks)

Epoch 1/50


2021-11-24 03:22:20.808066: I tensorflow/stream_executor/cuda/cuda_dnn.cc:366] Loaded cuDNN version 8101
2021-11-24 03:22:21.490665: W tensorflow/core/common_runtime/bfc_allocator.cc:275] Allocator (GPU_0_bfc) ran out of memory trying to allocate 898.00MiB with freed_by_count=0. The caller indicates that this is not a failure, but may mean that there could be performance gains if more memory were available.
2021-11-24 03:22:21.490714: W tensorflow/core/common_runtime/bfc_allocator.cc:275] Allocator (GPU_0_bfc) ran out of memory trying to allocate 898.00MiB with freed_by_count=0. The caller indicates that this is not a failure, but may mean that there could be performance gains if more memory were available.
2021-11-24 03:22:21.490749: W tensorflow/core/common_runtime/bfc_allocator.cc:275] Allocator (GPU_0_bfc) ran out of memory trying to allocate 2.28GiB with freed_by_count=0. The caller indicates that this is not a failure, but may mean that there could be performance gains if more me

75/75 [==============================] - 16s 160ms/step - loss: 40.2950 - accuracy: 0.6267 - val_loss: 21.3334 - val_accuracy: 0.5033
Epoch 2/50
75/75 [==============================] - 13s 172ms/step - loss: 18.1431 - accuracy: 0.7000 - val_loss: 16.1599 - val_accuracy: 0.5229
Epoch 3/50
75/75 [==============================] - 13s 174ms/step - loss: 15.4317 - accuracy: 0.7333 - val_loss: 14.3890 - val_accuracy: 0.8954
Epoch 4/50
75/75 [==============================] - 13s 174ms/step - loss: 14.5500 - accuracy: 0.7583 - val_loss: 14.0779 - val_accuracy: 0.8366
Epoch 5/50
75/75 [==============================] - 13s 173ms/step - loss: 14.1902 - accuracy: 0.7950 - val_loss: 14.0114 - val_accuracy: 0.8987
Epoch 6/50
75/75 [==============================] - 13s 172ms/step - loss: 14.0250 - accuracy: 0.8100 - val_loss: 13.7243 - val_accuracy: 0.9118
Epoch 7/50
75/75 [==============================] - 13s 173ms/step - loss: 13.9223 - accuracy: 0.8267 - val_loss: 13.8853 - val_accuracy: 0.8

## 아무 전처리도 하지 않은 사진 학습
학습 방법은 위와 같음

In [2]:
def getImage(mode, size):
    images = []
    labels = []
    for idx, now in enumerate(['Mask', 'Non Mask']):
        default_path = os.path.join(default_data_path, mode, now)
        img_name_list = os.listdir(default_path)

        for img_name in img_name_list:
            img = cv2.imread(os.path.join(default_path, img_name))
            img = cv2.resize(img, size)
            images.append(img)
            labels.append(idx)
    return np.asarray(images) / 225, np.asarray(labels)

In [10]:
train_data, train_y = getImage('Train', (224,224))
val_data, val_y = getImage('Validation', (224,224))

model = get_vgg16()
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])
callbacks = [keras.callbacks.EarlyStopping(monitor='accuracy', mode='max', patience=5),
                 keras.callbacks.ModelCheckpoint(filepath='./data/model/best_model.h5', monitor='accuracy', save_best_only=True)]
model.fit(train_data, train_y, epochs=50, batch_size=8,
                    validation_data=(val_data, val_y), shuffle=True, callbacks=callbacks)

Epoch 1/50


2021-11-24 03:38:50.166478: I tensorflow/stream_executor/cuda/cuda_dnn.cc:366] Loaded cuDNN version 8101
2021-11-24 03:38:50.832364: W tensorflow/core/common_runtime/bfc_allocator.cc:275] Allocator (GPU_0_bfc) ran out of memory trying to allocate 898.00MiB with freed_by_count=0. The caller indicates that this is not a failure, but may mean that there could be performance gains if more memory were available.
2021-11-24 03:38:50.832411: W tensorflow/core/common_runtime/bfc_allocator.cc:275] Allocator (GPU_0_bfc) ran out of memory trying to allocate 898.00MiB with freed_by_count=0. The caller indicates that this is not a failure, but may mean that there could be performance gains if more memory were available.
2021-11-24 03:38:50.832447: W tensorflow/core/common_runtime/bfc_allocator.cc:275] Allocator (GPU_0_bfc) ran out of memory trying to allocate 2.28GiB with freed_by_count=0. The caller indicates that this is not a failure, but may mean that there could be performance gains if more me

75/75 [==============================] - 18s 190ms/step - loss: 39.9828 - accuracy: 0.7117 - val_loss: 19.2971 - val_accuracy: 0.6961
Epoch 2/50
75/75 [==============================] - 13s 172ms/step - loss: 17.6177 - accuracy: 0.7767 - val_loss: 15.7407 - val_accuracy: 0.7386
Epoch 3/50
75/75 [==============================] - 13s 174ms/step - loss: 15.0171 - accuracy: 0.8383 - val_loss: 14.6664 - val_accuracy: 0.9641
Epoch 4/50
75/75 [==============================] - 13s 172ms/step - loss: 14.2527 - accuracy: 0.8550 - val_loss: 13.8301 - val_accuracy: 0.9346
Epoch 5/50
75/75 [==============================] - 10s 139ms/step - loss: 14.0582 - accuracy: 0.8550 - val_loss: 15.9518 - val_accuracy: 0.5000
Epoch 6/50
75/75 [==============================] - 13s 173ms/step - loss: 13.9357 - accuracy: 0.8717 - val_loss: 13.5063 - val_accuracy: 0.9608
Epoch 7/50
75/75 [==============================] - 13s 174ms/step - loss: 13.7629 - accuracy: 0.8967 - val_loss: 13.6128 - val_accuracy: 0.9

## 두 모델 성능 비교

In [8]:
test_data_sobel, test_y_sobel = getSobelImage('Test', (224, 224))
test_data, test_y = getImage('Test', (224,224))

In [11]:
model_sobel = keras.models.load_model('./data/model/best_model_sobel.h5')
model = keras.models.load_model('./data/model/best_model.h5')

In [12]:
print('소벨 모델 정확도:', model_sobel.evaluate(test_data_sobel, test_y_sobel)[1])
print('일반 모델 정확도:', model.evaluate(test_data, test_y)[1])

4/4 [==============================] - 1s 152ms/step - loss: 13.6040 - accuracy: 0.8800
소벨 모델 정확도: 0.8799999952316284
4/4 [==============================] - 1s 153ms/step - loss: 13.8823 - accuracy: 0.7800
일반 모델 정확도: 0.7799999713897705


[레포 보러가기](https://github.com/K1A2/mask_detection_vgg16_sobel/tree/master)